# Calculating Average Travel Time to Other Destination Categories
- Create a feature class for each destination type (healthcare, higher education, essential services) that contains the point locations, types, and capacity data.
- For each TAZ, find out how many (and ideally which) destinations of each type are within a buffer of the TAZ centroid.
- Flag each TAZ with whether it meets the threshold for the number of destinations within a buffer around the TAZ centroid for the type of destination.
- Use model skims for travel times from TAZ centroid to TAZ centroid for all of the following modes: Drive (SOV as proxy), Transit, Rapid Transit, Bus, Walk.
- Then pick the minimum time to reach a flagged TAZ and weight by TAZ population and use that for calculation of the average travel time for MPO.
- Then flag origin TAZs as whether their minimum distance to a flagged TAZ is within the average threshold for MPO.


Environment: base_py_37_omx_geop


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
import openmatrix as omx

In [6]:
##Import all the CSVs that contain skims 
Walk_skim= omx.open_file('M:/LRTP/LRTP_AvgTT/out/Walk_skim.omx','r')
#Import AM
DAT_Boat_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_Boat_tr_skim.omx','r')
DAT_CommRail_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_CommRail_tr_skim.omx','r')
DAT_LocalBus_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_LocalBus_tr_skim.omx','r')
DAT_RapidTransit_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_RapidTransit_tr_skim.omx','r')
WAT_Transit_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM\WAT_for_All_tr_skim.omx','r')
SOV_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/SOV_skim.omx','r')

#put AM CSVs into Dictionary
skims_AM = {'DAT_Boat':DAT_Boat_skim_AM, 'DAT_CR':DAT_CommRail_skim_AM, 
            'DAT_LB':DAT_LocalBus_skim_AM, 'DAT_RT':DAT_RapidTransit_skim_AM, 
            'WAT_TR':WAT_Transit_skim_AM, 'SOV':SOV_skim_AM, 'Walk': Walk_skim}

#Import MD
DAT_Boat_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_Boat_tr_skim.omx','r')
DAT_CommRail_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_CommRail_tr_skim.omx','r')
DAT_LocalBus_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_LocalBus_tr_skim.omx','r')
DAT_RapidTransit_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_Rapid_Transit_tr_skim.omx','r')
WAT_Transit_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\WAT_for_All_tr_skim.omx','r')
SOV_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\SOV_skim.omx','r')

#put MD CSVs into Dictionary
skims_MD = {'DAT_Boat':DAT_Boat_skim_MD, 'DAT_CR':DAT_CommRail_skim_MD, 
            'DAT_LB':DAT_LocalBus_skim_MD, 'DAT_RT':DAT_RapidTransit_skim_MD, 
            'WAT_TR':WAT_Transit_skim_MD, 'SOV':SOV_skim_MD, 'Walk': Walk_skim}

In [22]:
##Import TAZ tables for Essential Services, Healthcare, and Higher Ed. 
dest_TAZs = pd.read_csv(r'M:/LRTP/LRTP_AvgTT/MPO_TAZ_ES_1m2.csv',header=0, sep=',',
                        usecols=['taz', 'ES_FLAG_ALL','HIED_Count5', 'HLTH_COUNT1m'])
#create table with column for each destination type that flags (0,1) which TAZs pass the destination threshold
dest_TAZs['HIED_FLAG']=[1 if x >=1 else 0 for x in dest_TAZs['HIED_Count5']]
dest_TAZs['HLTH_FLAG']=[1 if x >=1 else 0 for x in dest_TAZs['HLTH_COUNT1m']]

dest_TAZs

,taz,ES_FLAG_ALL,HIED_Count5,HLTH_COUNT1m,HIED_FLAG,HLTH_FLAG
0,754,1,33,0,1,0
1,1688,0,2,0,1,0
2,89,1,45,17,1,1
3,1385,1,2,0,1,0
4,727,1,39,2,1,1
...,...,...,...,...,...,...
1896,2456,0,0,0,0,0
1897,2459,0,0,0,0,0
1898,2457,0,0,0,0,0
1899,2460,0,0,0,0,0


In [15]:
##Sum the CSVs by Time of Day and Mode (loop through both AM and MD dictionaries)
#Turn TAZ ID into Index so it will not sum (for all tables)
#Sum WAT with DAT 

#Sum by Mode (within TOD)

#Create new table in dictionary: All Transit
	#take the average - weight it by # of people taking each type of transit



In [ ]:
##Calculate Averages
	#weighted average of all the travel times from TAZs to TAZs that pass the destination threshold (flagged), weighted by population of the origin TAZ.
	#do this for all modes and TOD - should end up with a number each for SOV, All Transit, Rapid Transit, Local Bus, Commuter Rail, Boat, and Walking for AM and MD.


In [ ]:
##Make Flag Tables
#if the TAZ has access to a destination TAZ within the MPO average for that mode, flag.
	#How do I make this into a table?  - maybe copy the TAZ to TAZ table and clear its contents, then just go down the column of all the destination TAZs and query the skim table for that mode and TOD.
	#where column name is in list (aka field in the table that flags destination TAZs for each type)

